한빛아카데미
https://www.hanbit.co.kr/academy/books/new_book_list.html?page=3

In [35]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm #반복문에서 진행상황을 시각화해주는데 사용되는 'tqdm'

In [38]:
#! pip install tqdm : 진행사항을 100% bar형태로 알려줌

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 78.3/78.3 kB ? eta 0:00:00


In [5]:
base_url='https://www.hanbit.co.kr' #url 분리
main_url='/academy/books/new_book_list.html'
url=f'{base_url}{main_url}?page=1'
res=requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [6]:
lis=soup.select('.sub_book_list')
print(len(lis)) #한페이지당 책이 20개씩

20


In [7]:
li=lis[0] #첫번째책 하나만 뽑기
li

<li class="sub_book_list">
<div class="view_box">
<span class="view_box_block">
<span class="img_hidden_a">
<span class="pop_quick_menu">
<a href="/academy/books/book_view.html?p_code=B3329728344"><img alt="상세보기" src="https://www.hanbit.co.kr/images/common/icon_view_detail.png"/></a>
</span>
<span class="price">32,000<span>원</span></span>
<span class="pop_quick_bg" onclick="location='/academy/books/book_view.html?p_code=B3329728344'"></span>
</span>
<!-- 예약도서인 경우 -->
<!-- ebook이 도서인 경우 -->
<img alt="" class="thumb" src="/data/books/B3329728344_m.jpg"/>
</span>
<div class="info">
<p class="book_tit"><a href="/academy/books/book_view.html?p_code=B3329728344">IT@CookBook, 예제로 배우는 회로이론</a></p>
<p class="book_writer">김명식 </p>
</div>
</div>
</li>

In [8]:
sub_href=li.select_one('.book_tit>a')['href'] #서브페이지 만들기. 속성값 뽑기
sub_href

'/academy/books/book_view.html?p_code=B3329728344'

In [9]:
#서브페이지에서 정보를 추출
sub_url=base_url+sub_href
sub_res=requests.get(sub_url)
sub_soup=BeautifulSoup(sub_res.text, 'html.parser')

In [11]:
title=sub_soup.select_one('.store_product_info_box>h3').get_text().strip()
title

'IT@CookBook, 예제로 배우는 회로이론'

In [16]:
#책 정보의 딕셔너리 생성
sub_lis=sub_soup.select('.info_list > li')
info_dict={}
for sub_li in sub_lis[:-1]: #마지막 li는 제외
    key=sub_li.find('strong').get_text().strip()
    value=sub_li.find('span').get_text().strip()
    info_dict[key[:-2]]=value #키값의 ':''을 없애기 위해서
info_dict

{'저자': '김명식',
 '출간': '2023-12-10',
 '페이지': '696 쪽',
 'ISBN': '9791156640042',
 '물류코드': '40004'}

In [17]:
li=lis[10] #역자
li

<li class="sub_book_list">
<div class="view_box">
<span class="view_box_block">
<span class="img_hidden_a">
<span class="pop_quick_menu">
<a href="/academy/books/book_view.html?p_code=B9678437685"><img alt="상세보기" src="https://www.hanbit.co.kr/images/common/icon_view_detail.png"/></a>
</span>
<span class="price">21,000<span>원</span></span>
<span class="pop_quick_bg" onclick="location='/academy/books/book_view.html?p_code=B9678437685'"></span>
</span>
<!-- 예약도서인 경우 -->
<!-- ebook이 도서인 경우 -->
<img alt="" class="thumb" src="/data/books/B9678437685_m.jpg"/>
</span>
<div class="info">
<p class="book_tit"><a href="/academy/books/book_view.html?p_code=B9678437685">STEM CookBook, TAO 해석학 Ⅱ(4판)</a></p>
<p class="book_writer">Terence Tao </p>
</div>
</div>
</li>

In [19]:
price=int(sub_soup.select_one('.pbr>strong').string.replace(',',''))
price

32000

In [26]:
data=[]
for li in lis:
    sub_href=li.select_one('.book_tit > a')['href']
    sub_url=base_url+sub_href
    sub_res=requests.get(sub_url)
    sub_soup=BeautifulSoup(sub_res.text, 'html.parser')
    title=sub_soup.select_one('.store_product_info_box>h3').get_text().strip()
    sub_lis=sub_soup.select('.info_list > li')
    info_dict={}
    for sub_li in sub_lis[:-1]:
        key=sub_li.find('strong').get_text().strip()
        value=sub_li.find('span').get_text().strip()
        info_dict[key[:-2]]=value
    price=int(sub_soup.select_one('.pbr > strong').string.replace(',',''))

#info_dict에서 저자 역자 출간일 페이지 추출
    author=', '.join(info_dict['저자'].split(' , '))
    translator=info_dict['번역'].replace(' ,',',') if '번역' in info_dict.keys() else ' ' #역자 없으면 공백
    date=info_dict['출간']
    page=int(info_dict['페이지'][:-2])

    data.append({'제목':title, '저자':author,'역자':translator,'출간일':date,'페이지수':page, '가격':price})

In [28]:
df=pd.DataFrame(data)
df.head()

,제목,저자,역자,출간일,페이지수,가격
0,"IT@CookBook, 예제로 배우는 회로이론",김명식,,2023-12-10,696,32000
1,4차 산업혁명 시대의 창업실무(2판),"윤두열, 이정이",,2023-12-08,436,29000
2,"IT CookBook, 쉽게 배우는 JSP 웹 프로그래밍(2판)",송미영,,2023-12-08,676,30000
3,"IT CookBook, 핵심이 보이는 신호 및 시스템(2판)",이철희,,2023-12-08,688,33000
4,"STEM CookBook, 수학의 깊이를 더하는 『STEIN 복소해석학』 강의노트",김영원,,2023-11-20,200,22000


In [34]:
#전체 페이지에서 정보 추출하기
data1=[]
for page in tqdm(range(1,25)): #24페이지까지
    url=f'{base_url}{main_url}?page={page}'
    res=requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    for li in lis:
        sub_href=li.select_one('.book_tit > a')['href']
        sub_url=base_url+sub_href
        sub_res=requests.get(sub_url)
        BeautifulSoup(sub_res.text, 'html.parser')
        title=sub_soup.select_one('.store_product_info_box>h3').get_text().strip()
        sub_lis=sub_soup.select('.info_list > li')
        info_dict={}
        for sub_li in sub_lis[:-1]:
            key=sub_li.find('strong').get_text().strip()
            value=sub_li.find('span').get_text().strip()
            info_dict[key[:-2]]=value
        price=int(sub_soup.select_one('.pbr > strong').string.replace(',',''))

#info_dict에서 저자 역자 출간일 페이지 추출
        author=', '.join(info_dict['저자'].split(' , '))
        translator=info_dict['번역'].replace(' ,',',') if '번역' in info_dict.keys() else ' ' #역자 없으면 공백
        date=info_dict['출간']
        page=int(info_dict['페이지'][:-2])

        data1.append({'제목':title, '저자':author,'역자':translator,'출간일':date,'페이지수':page, '가격':price})

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [02:13<00:00,  5.56s/it]
